In [3]:
#导入书籍
import pandas as pd
# 有的环境配置下read_table出错，因此改用read_csv
raw = pd.read_csv("金庸-射雕英雄传txt精校版.txt",
                  names = ['txt'], sep ='aaa', encoding ="GBK" ,engine='python')
# 章节判断用变量预处理
def m_head(tmpstr):
    return tmpstr[:1]
def m_mid(tmpstr):
    return tmpstr.find("回 ")
raw['head'] = raw.txt.apply(m_head)
raw['mid'] = raw.txt.apply(m_mid)
raw['len'] = raw.txt.apply(len)
# 章节判断
chapnum = 0
for i in range(len(raw)):
    if raw['head'][i] == "第" and raw['mid'][i] > 0 and raw['len'][i] < 30 :
        chapnum += 1
    if chapnum >= 40 and raw['txt'][i] == "附录一：成吉思汗家族" :
        chapnum = 0
    raw.loc[i, 'chap'] = chapnum  
# 删除临时变量
del raw['head']
del raw['mid']
del raw['len']
raw.head(0)
rawgrp = raw.groupby('chap')
chapter = rawgrp.sum()##.agg(sum) # 只有字符串列的情况下，sum函数自动转为合并字符串
chapter = chapter[chapter.index != 0]
#第三四段内容
chapter = chapter.txt[3]+chapter.txt[4]

In [8]:
import jieba
import jieba.analyse

# 注意：函数是在使用默认的TFIDF模型进行分析！
jieba.analyse.extract_tags(chapter, withWeight = True) # 要求返回权重值

[('铁木真', 0.13626546781284937),
 ('郭靖', 0.0817636657844646),
 ('柯镇恶', 0.0755037947551579),
 ('韩小莹', 0.06856091707652268),
 ('哲别', 0.06422161852737568),
 ('拖雷', 0.046356070188021785),
 ('韩宝驹', 0.045996564620958254),
 ('朱聪', 0.04165726607181125),
 ('梅超风', 0.04165726607181125),
 ('丘处机', 0.03796879707411252),
 ('全金发', 0.037317967522664246),
 ('张阿生', 0.03471438839317604),
 ('陈玄风', 0.03471438839317604),
 ('段天德', 0.03384652868334664),
 ('完颜洪熙', 0.032978668973517235),
 ('李萍', 0.03164054486796371),
 ('博尔', 0.029819683306482762),
 ('蒙古', 0.02861885878071579),
 ('王罕', 0.026237213717023596),
 ('江南七怪', 0.02603579129488203)]

In [10]:
# 应用自定义词典改善分词效果
jieba.load_userdict('金庸小说词库.txt') # dict为自定义词典的路径

# 在TFIDF计算中直接应用停用词表
jieba.analyse.set_stop_words('停用词.txt')

TFres = jieba.analyse.extract_tags(chapter, withWeight = True)
TFres[:10]

[('铁木真', 0.1549749685536658),
 ('郭靖', 0.10745507449253633),
 ('柯镇恶', 0.085870605412178),
 ('朱聪', 0.07896147624108321),
 ('韩小莹', 0.07797445778806968),
 ('哲别', 0.07303936552300197),
 ('拖雷', 0.05272084435601057),
 ('韩宝驹', 0.05231197800971763),
 ('梅超风', 0.04737688574464993),
 ('丘处机', 0.04318198313209214)]

In [11]:
# 使用自定义TF-IDF频率文件
jieba.analyse.set_idf_path("idf.txt.big")
TFres1 = jieba.analyse.extract_tags(chapter, withWeight = True)
TFres1[:10]

[('铁木真', 0.17470226618339663),
 ('郭靖', 0.10264991911340818),
 ('柯镇恶', 0.085870605412178),
 ('朱聪', 0.07896147624108321),
 ('韩小莹', 0.07797445778806968),
 ('一个', 0.07402638397601552),
 ('哲别', 0.07303936552300197),
 ('拖雷', 0.05922110718081241),
 ('韩宝驹', 0.05231197800971763),
 ('孩子', 0.05231197800971763)]